# AI Writing Assistant
This GPT-powered Writing Assistant helps users enhance their writing by using three intelligent tools
 1. Magic world- Generate fancy words
 2. Grammer Check - Check Grammatical mistakes
 3. Plot Idea Generate- Generate interesting plot ideas based on genra or
 theme.
### It uses few-shot prompt engineering to teach the model how to use tools based on real user examples. When a user sends a message, the assistant decides whether to respond directly or call a tool function to generate a more helpful answer.

## Installing openai version 0.28 and Import the openAI API


In [1]:
!pip install openai==0.28

#import the key form colab userdata
import openai
from google.colab import userdata
openai.api_key =userdata.get('chatKey')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.72.0
    Uninstalling openai-1.72.0:
      Successfully uninstalled openai-1.72.0


### Defining three different functions for handeling three intelligence

In [2]:
import random

def magic_word(word: str) -> str:
    # Slightly more varied mocked synonyms mock base
    synonyms = {
        "happy": ["joyful", "cheerful", "content", "delighted"],
        "sad": ["unhappy", "downcast", "melancholy", "gloomy"],
        "very big": ["enormous", "gigantic", "massive", "colossal"],
        "fast": ["quick", "swift", "rapid", "speedy"]
    }
    matches = synonyms.get(word.lower(), ["(no known synonyms found)"])
    return f"Some synonyms for '{word}' are: {', '.join(matches)}."

def plot_idea_generator(genre: str) -> str:
    # Rotating ideas for fun mock base
    ideas = {
        "sci-fi": [
            "In a world ruled by AI, one human hacker uncovers a forgotten truth.",
            "After Earth becomes uninhabitable, a child on Mars discovers ancient alien tech."
        ],
        "romance": [
            "Two rival writers fall in love while unknowingly collaborating online.",
            "A time traveler returns to the 1800s to find their soulmate."
        ],
        "fantasy": [
            "A kitchen maid discovers she’s the heir to a forgotten dragon throne.",
            "A cursed forest holds the key to saving the realm—but at a great cost."
        ]
    }
    genre_key = genre.lower()
    if genre_key in ideas:
        idea = random.choice(ideas[genre_key])
        return f"A {genre} story idea: {idea}"
    return f"Sorry, I don't have story ideas for the '{genre}' genre right now."

def grammar_checker(text: str) -> str:
    # Simple mock correction logic mock base
    common_errors = {
        "He go to school": "He goes to school",
        "She walk": "She walks",
        "buy apples": "bought apples",
        "I is": "I am"
    }
    corrected = text
    for error, fix in common_errors.items():
        if error in corrected:
            corrected = corrected.replace(error, fix)
    return f"Revised: '{corrected}'"


### Defining tools and some few-shot prompts

In [7]:
#---tool functions for open ai
tools = [
    {
        "type": "function",
        "function": {
            "name": "magic_word",
            "description": "Find synonyms for a word to enhance vocabulary.",
            "parameters": {
                "type": "object",
                "properties": {
                    "word": {"type": "string", "description": "The word to look up synonyms for."}
                },
                "required": ["word"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "plot_idea_generator",
            "description": "Generate a story plot idea based on genre or theme.",
            "parameters": {
                "type": "object",
                "properties": {
                    "genre": {"type": "string", "description": "Genre or theme (e.g., fantasy, romance, dystopian)."}
                },
                "required": ["genre"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "grammar_checker",
            "description": "Check grammar and suggest improvements for a paragraph.",
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {"type": "string", "description": "The paragraph to analyze and improve."}
                },
                "required": ["text"]
            }
        }
    }
]

# --- Few-shot Prompt Examples

few_shot_examples = [
    {
        "role": "user",
        "content": "Give me a story idea in the horror genre."
    },
    {
        "role": "assistant",
        "tool_calls": [
            {
                "id": "call_1",
                "type": "function",
                "function": {
                    "name": "plot_idea_generator",
                    "arguments": "{\"genre\": \"horror\"}"
                }
            }
        ]
    },
    {
        "role": "tool",
        "tool_call_id": "call_1",
        "content": "A horror story idea: A family moves into a remote village where no one speaks—but everyone listens."
    },
    {
        "role": "user",
        "content": "What’s a better word for 'very big'?"
    },
    {
        "role": "assistant",
        "tool_calls": [
            {
                "id": "call_2",
                "type": "function",
                "function": {
                    "name": "thesaurus_lookup",
                    "arguments": "{\"word\": \"very big\"}"
                }
            }
        ]
    },
    {
        "role": "tool",
        "tool_call_id": "call_2",
        "content": "Some synonyms for 'very big' are: enormous, gigantic, massive, colossal."
    },
    {
        "role": "user",
        "content": "Can you fix this? 'She walk to the store and buy apples.'"
    },
    {
        "role": "assistant",
        "tool_calls": [
            {
                "id": "call_3",
                "type": "function",
                "function": {
                    "name": "grammar_checker",
                    "arguments": "{\"text\": \"She walk to the store and buy apples.\"}"
                }
            }
        ]
    },
    {
        "role": "tool",
        "tool_call_id": "call_3",
        "content": "Revised: 'She walks to the store and bought apples.'"
    }
]


### Initialize gtp-4o-mini model for generating responces

In [8]:
import json

#--- function for handle tool calls
def chat_with_tools(user_message):
    messages = [
        {"role": "system", "content": "You are a helpful writing assistant. You help users with grammar, vocabulary, and writing ideas. Use tools when needed."}
    ] + few_shot_examples + [
        {"role": "user", "content": user_message}
    ]

    while True:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=messages,
                tools=tools,
                tool_choice="auto"
            )
        except Exception as e:
            return f"Error: {str(e)}"

        message = response.choices[0].message
        messages.append(message)

        if not message.get("tool_calls"):
            return message.get("content")

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)

            if function_name == "magic_word":
                result = magic_word(function_args.get("word"))
            elif function_name == "plot_idea_generator":
                result = plot_idea_generator(function_args.get("genre"))
            elif function_name == "grammar_checker":
                result = grammar_checker(function_args.get("text"))
            else:
                result = "Unknown function."

            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result
            })


### lets try the assistant

In [9]:
query1 = "Suggest a story plot in the sci-fi genre."
print(f"Q: {query1}\nA: {chat_with_tools(query1)}\n")


Q: Suggest a story plot in the sci-fi genre.
A: Here’s a sci-fi story idea: After Earth becomes uninhabitable, a child living on Mars discovers ancient alien technology that holds the key to humanity’s survival.



In [10]:
query2 = "What’s a fancier word for 'happy'?"
print(f"Q: {query2}\nA: {chat_with_tools(query2)}\n")

Q: What’s a fancier word for 'happy'?
A: Fancier words for "happy" include: joyful, cheerful, content, and delighted.



In [12]:
query3 = "Can you correct this: 'He go to school every day.'"
print(f"Q: {query3}\nA: {chat_with_tools(query3)}\n")

Q: Can you correct this: 'He go to school every day.'
A: The corrected sentence is: **"He goes to school every day."**

